<a href="https://colab.research.google.com/github/vitroid/PythonTutorials/blob/master/Pending/LCG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 線形合同法の逆算

$X_{n+1}=(A X_n + B) \textrm{ mod } M$
の生成した数列から、逆にA, B, Mを推定できるか?

In [11]:
import matplotlib.pyplot as plt
import numpy as np
def rand(n=10, A=1103515245, B=12345, M=2**32):
    x = 8
    for i in range(n):
        x = (A*x+B) % M
        yield x


x = np.array([i for i in rand(100)])
x

array([ 238199713, 3908102598, 2867989895, 1184437428, 1790236893,
       1627511890,  408419107, 1275906080,  791356889, 2579327902,
       2258970239, 2873509452,   18826901, 1846584234, 2926554011,
       1622637368, 2600122129, 1219317878, 4052969079, 1006965476,
       2588062541,  999685634,  164492051, 3228727632, 4120131401,
         74053710,   85634415, 2685262460, 3596615429,  158419802,
       2882449803, 3181126248, 1382450817,  459850534,  745855847,
       1631911188, 1089735101, 2282613682, 2842281731,  201250432,
        668763321, 3379237118,   70766687,  364567212, 4231508853,
       3722657546,   45290363,   38860184,  853348849,  352207318,
       2849336407,  959173444, 1553980461, 3707020642, 1784313587,
       2343886768, 1246432809, 3707483566, 2065403727, 3061860060,
       1952277477, 1750171322, 2100023659, 3375953096,  579164513,
        369675398, 1169430855,  558667124, 2396859037,  160770834,
       1770980067, 1666684128,   15127449,  941024862, 3869567

In [12]:
2**32

4294967296

## 1. A, Mがわかっていて、Bを予測する。

$$X_1=(A X_0 + B) \mod M$$
$$(X_1-A X_0)\mod M=B$$



In [15]:
A = 1103515245
M = 2**32
(x[1:] - A*x[:-1]) % M


array([12345, 12345, 12345, 12345, 12345, 12345, 12345, 12345, 12345,
       12345, 12345, 12345, 12345, 12345, 12345, 12345, 12345, 12345,
       12345, 12345, 12345, 12345, 12345, 12345, 12345, 12345, 12345,
       12345, 12345, 12345, 12345, 12345, 12345, 12345, 12345, 12345,
       12345, 12345, 12345, 12345, 12345, 12345, 12345, 12345, 12345,
       12345, 12345, 12345, 12345, 12345, 12345, 12345, 12345, 12345,
       12345, 12345, 12345, 12345, 12345, 12345, 12345, 12345, 12345,
       12345, 12345, 12345, 12345, 12345, 12345, 12345, 12345, 12345,
       12345, 12345, 12345, 12345, 12345, 12345, 12345, 12345, 12345,
       12345, 12345, 12345, 12345, 12345, 12345, 12345, 12345, 12345,
       12345, 12345, 12345, 12345, 12345, 12345, 12345, 12345, 12345])

## 2. B, Mがわかっていて、Aを予測する。

$$X_1=(A X_0 + B) \mod M$$
$$X_2=(A X_1 + B) \mod M$$
引き算してBを消す。
$$X_2-X_1\mod M=A(X_1-X_0)\mod M$$
$$(X_2-X_1)(X_1-X_0)^{-1}\mod M=A\mod M$$
ただし、$^{-1}$は逆数じゃなくて、法Mでの逆元。

逆元とは、整数$X$に対して、$X X^{-1}\mod M=1$となるような整数$X^{-1}$のこと、だと思う。
(存在しない場合もありそうだが、ここではそれを心配する必要はない)

逆元の計算は、Python3.8ならpow関数で一発で求まるんだが、colabは3.7らしいぞ。まいった。



In [19]:
# Python3.8
((x[2] - x[1])*pow(int(x[1]-x[0]), -1, M)) % M

ValueError: ignored

In [20]:
!python -V

Python 3.7.13


In [21]:
# Python3.7

def xgcd(a, b):
    x0, y0, x1, y1 = 1, 0, 0, 1
    while b != 0:
        q, a, b = a // b, b, a % b
        x0, x1 = x1, x0 - q * x1
        y0, y1 = y1, y0 - q * y1
    return a, x0, y0

def modinv(a, m):
    g, x, y = xgcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m

((x[2] - x[1])*modinv(x[1]-x[0], M)) % M


1103515245

## 3. A, Bがわかっていて、Mを予測する。

$x_0$がとても小さい数であれば、$x_1=Ax_0+B$となり、$M$は推定できないが、それ以外なら、
$$Ax_0+B-x_1=n M$$
となるだろう。

In [27]:
A = 1103515245
B = 12345
X0 = x[:-1]
X1 = x[1:]

nM = A*X0+B - X1
print(np.gcd(nM[:-1], nM[1:]))

[  4294967296  17179869184  51539607552 240518168576   4294967296
   4294967296   4294967296   4294967296   4294967296   4294967296
   4294967296  17179869184  17179869184   4294967296   4294967296
   4294967296   4294967296   4294967296   4294967296   4294967296
   4294967296   8589934592   4294967296   4294967296 257698037760
  21474836480   4294967296  17179869184   8589934592  12884901888
  12884901888   4294967296   4294967296   4294967296   4294967296
   8589934592   8589934592   8589934592   4294967296   4294967296
   4294967296   8589934592  25769803776   4294967296   4294967296
   4294967296   4294967296   4294967296   4294967296   4294967296
   4294967296   4294967296   4294967296   8589934592   4294967296
   4294967296   4294967296  25769803776  12884901888   4294967296
   8589934592   8589934592   4294967296   4294967296  17179869184
   4294967296   4294967296   4294967296  12884901888  25769803776
  12884901888  12884901888   8589934592 124554051584   4294967296
   8589934

データがいくつかあれば、推定できる。

## Mだけわかっている場合

$B$が不明でも、$A$を求めることができたので、$A,B$は同時に不明でも構わない。

## $A,B,M$のいずれも不明な場合

まず$B$を消す。
$$X_1=(AX_0+B)\mod M$$
$$X_2=(AX_1+B)\mod M$$
$$(X_2-X_1)=A(X_1-X_0)\mod M$$
さらに$A$も消す。
$$(X_2-X_1)(X_1-X_0)^{-1}=A\mod M$$
$$(X_2-X_1)(X_1-X_0)^{-1} = (X_3-X_2)(X_2-X_1)^{-1}\mod M$$
逆元は$M$が不明だと計算できないので、両辺に$(X_1-X_0)(X_2-X_1)$を掛ける。
$$(X_2-X_1)(X_2-X_1)-(X_3-X_2)(X_1-X_0)=0\mod M$$
これで推定できるはず。






In [31]:
X3 = x[3:]
X2 = x[2:][:-1]
X1 = x[1:][:-2]
X0 = x[:-3]

nM = (X2-X1)**2 - (X3-X2)*(X1-X0)
print(np.gcd(nM[:-1], nM[1:]))

[    4294967296    17179869184     4294967296     4294967296
     4294967296     4294967296     4294967296     4294967296
     4294967296     4294967296    12884901888     4294967296
     4294967296     4294967296     8589934592     8589934592
     8589934592     8589934592   115964116992    38654705664
    12884901888     4294967296     4294967296    12884901888
     4294967296    21474836480     4294967296     4294967296
     4294967296    12884901888     8589934592   188978561024
     4294967296     4294967296     4294967296    34359738368
    21474836480     4294967296     4294967296     4294967296
   450971566080    64424509440    12884901888     8589934592
     4294967296     4294967296     4294967296    12884901888
   167503724544     4294967296     4294967296     4294967296
     4294967296     4294967296     4294967296     4294967296
     4294967296    12884901888     4294967296     4294967296
     4294967296     4294967296    12884901888     4294967296
     4294967296     4294

## まとめ

これらをまとめよう。

In [41]:
def xgcd(a, b):
    x0, y0, x1, y1 = 1, 0, 0, 1
    while b != 0:
        q, a, b = a // b, b, a % b
        x0, x1 = x1, x0 - q * x1
        y0, y1 = y1, y0 - q * y1
    return a, x0, y0

def modinv(a, m):
    g, x, y = xgcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m

def modinvs(A, m):
    return np.array([modinv(a, m) for a in A])

def predictM(x):
    X3 = x[3:]
    X2 = x[2:][:-1]
    X1 = x[1:][:-2]
    X0 = x[:-3]

    nM = (X2-X1)**2 - (X3-X2)*(X1-X0)
    M = np.gcd.reduce(nM)
    if not np.all(x<M):
        raise Exception('Inconsistent modulo.')
    return M

def predictA(x, M):
    X2 = x[2:][:-1]
    X1 = x[1:][:-2]
    X0 = x[:-3]
    A = ((X2 - X1)*modinvs(X1-X0, M)) % M
    if not np.all(A == A[0]):
        raise Exception('Inconsistent A')
    return A[0]

def predictB(x, M, A):
    X1 = x[1:][:-2]
    X0 = x[:-3]
    B = (X1 - A*X0) % M
    if not np.all(B == B[0]):
        raise Exception('Inconsistent B')
    return B[0]

M = predictM(x)
A = predictA(x, M)
B = predictB(x, M, A)
M, A, B

(4294967296, 1103515245, 12345)

In [43]:
s = (x[0]-B)*modinv(A,M)
s % M

8